In [4]:
import json
import pathlib

import numpy as np

from scripts.utils import *

TRACE = 'real'

TRACE_NAME = TRACE_NAMES[TRACE]
TRACE_DIR = TRACE_DIRS[TRACE]
TRACE_TYPE = TRACE_TYPES[TRACE]

OVERHEADS = [0.1]

DDL = 52
DDL_GAPS = 52 * 3600 // 600


In [5]:
overhead = 0.1
sliced_interval = 4.0
algorithm = 'loose_time_sliced'

slice_gap = sliced_interval * 3600 // 600


EXP_PATH = EXP_DIR / TRACE_NAME.format(overhead=overhead)
RESULT_PATH = RESULT_DIR / (TRACE_NAME.format(overhead=overhead) + '.json')
TRACE_PATH = TRACE_DIR / TRACE_TYPE
file_name = f'-trace-{TRACE_TYPE}-ddl=*-dur=48.0-over={overhead}'

alg_trace = list(EXP_PATH.glob(f'{algorithm}_{sliced_interval:.1f}h{file_name}'))[0]

trace = json.load(alg_trace.open('r'))
request_traces = [[int(i['RequestType'] - 1) for i in h[1:]] for h in trace['history']]
for i, r in enumerate(request_traces):
    r = r[:DDL_GAPS]
    if len(r) < DDL_GAPS:
        r += [0] * (DDL_GAPS - len(r))
    request_traces[i] = r
request_traces = np.array(request_traces)

In [10]:
slice_gap = int(sliced_interval * 3600 // 600)
sliced = request_traces.reshape(len(request_traces), -1, slice_gap)

In [11]:
(sliced > 1).sum(-1)

array([[ 0,  2, 18, ...,  0, 14,  0],
       [ 0,  3,  8, ...,  0,  0,  0],
       [ 0,  0,  0, ...,  9, 24, 12],
       ...,
       [ 0,  0,  0, ...,  0,  0,  0],
       [ 1,  0,  0, ..., 11,  6, 23],
       [ 0,  0,  0, ...,  0,  0,  0]])